In [60]:
import pyodbc
import pandas as pd

In [61]:
def connect_to_db(driver, server, database, username, password):
    """
    String para conectar a la base de datos SQL y Sypnase Analytics.
    """
    conn_str = (
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"Uid={username};"
        f"Pwd={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
        f"Connection Timeout=30;"
    )
    return pyodbc.connect(conn_str)

In [62]:
def extract_data(conn, query):
    """
    Extraer los datos de la base de datos SQL - Gestion de Clientes Seguros XYZ
    """
    return pd.read_sql(query, conn)

In [63]:
def validate_phone(phone):
    """
    Validar que el número teléfonico sea valido
    """
    if phone.isdigit() and len(phone) == 10:
        return phone
    else:
        return "DatoIncorrecto"

In [64]:
def clean_transform_data(df):
    """
    Función para Transformar y Limpiar los datos de la fuente de origen para cargarlo al DataWareHouse
    """
    # Borrar Columnas cuyo nombre viene NULL
    df = df[df['Nombre'].notnull()]

    # Verificar los datos que llegan de otras columnas NULL y asignar un nuevo valor como se detalla acontinuación.
    df['Apellido'].fillna('SinDato', inplace=True)
    df['Direccion'].fillna('SinDato', inplace=True)
    df['Telefono'] = df['Telefono'].apply(validate_phone)
    df['EstadoCivil'].fillna('SinDato', inplace=True)
    df['Profesion'].fillna('SinDato', inplace=True)
    df['IngresoAnual'].fillna(0, inplace=True)
    df['IngresoAnual'] = pd.to_numeric(df['IngresoAnual'], errors='coerce')
    df['IngresoAnual'] = df['IngresoAnual'].apply(lambda x: x if x >= 0 else 0)  # Reemplazar los valores Negativos con Cero 
    df['Estado'].fillna('SinDato', inplace=True)

    # Normalizar los textos en la base de datos
    df['Nombre'] = df['Nombre'].str.strip().str.title()
    df['Apellido'] = df['Apellido'].str.strip().str.title()
    df['Direccion'] = df['Direccion'].str.strip().str.title()
    df['EstadoCivil'] = df['EstadoCivil'].str.strip().str.capitalize()
    df['Profesion'] = df['Profesion'].str.strip().str.title()
    df['Estado'] = df['Estado'].str.strip().str.capitalize()

    # Crear una nueva columna que contenga la combinación del Nombre + Apellido
    df['NombreCompleto'] = df['Nombre'] + ' ' + df['Apellido']

    # En el caso de que se presenten duplicados con la identificación del cliente, se proceden a borrar
    df.drop_duplicates(subset='ClienteID', keep='last', inplace=True)

    return df

In [65]:
def detect_inconsistencies(df):
    """
    Detectar Inconsistencias en la Data.
    """
    inconsistencies = {
        'null_values': {},
        'invalid_phone': [],
        'negative_income': [],
        'duplicates': []
    }

    # Encontrar valores NULL
    inconsistencies['null_values'] = df.isnull().sum()

    # Validar numeros telefonicos
    invalid_phone_rows = df[df['Telefono'] == 'DatoIncorrecto']
    inconsistencies['invalid_phone'] = invalid_phone_rows.index.tolist()

    # Encontrar Ingresos Anuales negativos
    negative_income_rows = df[df['IngresoAnual'] < 0]
    inconsistencies['negative_income'] = negative_income_rows.index.tolist()

    # Encontrar datos duplicados basados en ClienteID
    duplicates = df[df.duplicated(subset='ClienteID', keep=False)]
    inconsistencies['duplicates'] = duplicates.index.tolist()

    return inconsistencies

In [66]:
def load_data(df, conn):
    """
    Función para cargar la data transformada en la tabla del datawarehouse
    """
    cursor = conn.cursor()
    for index, row in df.iterrows():
        cursor.execute("""
        INSERT INTO mart.DimClientes (ClienteID, NombreCompleto, Direccion, Telefono, FechaNacimiento, Sexo, EstadoCivil, Profesion, IngresoAnual, FechaRegistro, Estado)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
        row['ClienteID'], row['NombreCompleto'], row['Direccion'], row['Telefono'], row['FechaNacimiento'], row['Sexo'], row['EstadoCivil'], row['Profesion'], row['IngresoAnual'], row['FechaRegistro'], row['Estado'])
    conn.commit()

In [67]:
def main():
    # Datos de conexión a la base de datos del sistema de gestión de clientes
    client_db_driver = 'ODBC Driver 18 for SQL Server'
    client_db_server = 'tcp:segurosxyz.database.windows.net,1433'
    client_db_database = 'GestionClientesXYZ'
    client_db_username = 'sqladminuser2024'
    client_db_password = 'Segurosxyz.2024'

    # Datos de conexión al data warehouse en Azure Synapse Analytics
    dw_driver = 'ODBC Driver 18 for SQL Server'
    dw_server = 'tcp:workspacesegurosxyz.sql.azuresynapse.net,1433'
    dw_database = 'segurosxyz_sqlpool'
    dw_username = 'sqladminuser'
    dw_password = 'Segurosxyz.2024'

    # Conectar a la base de datos del sistema de gestión de clientes
    conn_clientes = connect_to_db(client_db_driver, client_db_server, client_db_database, client_db_username, client_db_password)
    query_clientes = "SELECT * FROM Clientes"
    df_clientes = extract_data(conn_clientes, query_clientes)

    # Limpiar y transformar los datos
    df_transformed = clean_transform_data(df_clientes)
    
    # Conectar al data warehouse en Azure Synapse Analytics
    conn_synapse = connect_to_db(dw_driver, dw_server, dw_database, dw_username, dw_password)
    load_data(df_transformed, conn_synapse)
    
    # Mensaje de confirmación
    print("Los datos se han copiado exitosamente.")

In [68]:
if __name__ == "__main__":
    main()

C:\Users\CATALINA\AppData\Local\Temp\ipykernel_21820\333626351.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\CATALINA\AppData\Local\Temp\ipykernel_21820\4224854718.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Apellido'].fillna('SinDato', inplace=True)
C:\Users\CATALINA\AppData\Local\Temp\ipykernel_21820\4224854718.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Direccion'].fillna('SinDato', inplace=

Los datos se han copiado exitosamente.
